
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

## Raw Table

In [0]:
from pyspark.sql.functions import col, when

# File location and type
file_location = "/FileStore/tables/fake_transactional_data_24__1_.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# Read CSV file into DataFrame
df_check = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Register DataFrame as a temporary view
df_check.createOrReplaceTempView("temp_table")

# Display the DataFrame
display(df_check)


### First column check - account

In [0]:
%sql
-- Query to check if all the entries end in .0
SELECT
  from_totally_fake_account
FROM
  temp_table
WHERE
  from_totally_fake_account NOT LIKE '%.0'

### Second column check - random generated account

In [0]:
%sql
SELECT
  DISTINCT
  from_totally_fake_account,
  to_randomly_generated_account
FROM
  temp_table
ORDER BY
  from_totally_fake_account

In [0]:
%sql
-- Focus on a single account 
SELECT
  from_totally_fake_account,
  to_randomly_generated_account
FROM
  temp_table
WHERE
 from_totally_fake_account = '1000.0'
ORDER BY
  to_randomly_generated_account

## Curated Table

In [0]:
from pyspark.sql.functions import col, when, lower, regexp_replace

# File location and type
file_location = "/FileStore/tables/fake_transactional_data_24__1_.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# Read CSV file into DataFrame
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Changing the column names
new_column_names = ["account_id", "purchase_amount", "purchase_type", "date_id"]
df = df.toDF(*new_column_names)

# Check if all values in 'account_id' end with '.0' and remove decimal part
df = df.withColumn("account_id_whole", 
                   when(col("account_id").substr(-2, 2) == ".0", 
                        col("account_id").cast("integer")).otherwise(col("account_id")))
df = df.drop("account_id").withColumnRenamed("account_id_whole", "account_id")

# Convert purchase amount column in to a float
df = df.withColumn("purchase_amount", df["purchase_amount"].cast("float"))

# Cleaning the purchase type column so its consistent
# Removing _ and replacing with spaces and lowercasing all text
df = df.withColumn("purchase_type",
                   when(df["purchase_type"].rlike("[A-Z_]"),
                        lower(regexp_replace(df["purchase_type"], "_", " "))).otherwise(df["purchase_type"]))

# Reorder columns
df = df.select("account_id", "purchase_amount", "purchase_type", "date_id")

# Display the DataFrame
display(df)
df.printSchema()



In [0]:
# Create a view or table

temp_table_name = "lloyds_raw_data"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `lloyds_raw_data`

In [0]:
%sql
DROP TABLE IF EXISTS lloyds_raw_data_table

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.

# Removes the table if its stored - Error handling
dbutils.fs.rm("dbfs:/user/hive/warehouse/lloyds_raw_data_table/", True)

# Saves the new table
raw_transactional_data = "lloyds_raw_data_table"
df.write.mode("overwrite").format("parquet").saveAsTable(raw_transactional_data)